In [1]:
import pandas as pd
import os
import geopandas as gpd

In [2]:
def investment_cost_calculator(r , n, maintenance, op, tf, cis):
    path = os.path.join((os.path.dirname(os.getcwd())), 'results')
    ci_dam = pd.read_excel(os.path.join(path, f'ci_{op}_{tf}_mosek_{10000}_{cis}.xlsx'), index_col = [0], header = [0])
    ci_failed = ci_dam[ci_dam['flood_fail'] == 0]
    osmids_toadapt = list(ci_failed['osmid'])
    input_path = os.path.join((os.path.dirname(os.path.dirname(os.getcwd()))), 'data')
    ci = gpd.read_parquet(os.path.join(input_path, 'substaions_nl33.parquet'))
    filtered_ci = ci[ci['osmid'].isin(osmids_toadapt)]
    filtered_cidam = ci_dam[ci_dam['osmid'].isin(osmids_toadapt)]
    filtered_cidam = filtered_cidam.drop(columns = ['geometry'])
    merged_df = pd.merge(filtered_ci, filtered_cidam, on='osmid')
    merged_df = merged_df.to_crs(32633)

    merged_df['perimeter'] = merged_df.geometry.length
    merged_df['constcost'] = (244.13 + 214.23*merged_df['depth']) * merged_df['perimeter'] # Model from retaining wall website
    merged_df['maintcost'] =  merged_df['constcost']  * maintenance # Assumption: 20% maintenance
    merged_df['totalcost'] =  merged_df['constcost']  + merged_df['maintcost']
    merged_df['annualcost'] = (merged_df['totalcost']  * r) / (1 - (1 + r) ** -n) # Annual cost
    investment = (merged_df['annualcost'].sum()) / 1000000 # in million Euros
    merged_df['surface_area'] = merged_df['depth'] * merged_df['perimeter'] # in sq.m
    surface_area = merged_df['surface_area'].sum()
    return investment

#### Benefits: Reduction in EAI

In [3]:
path = os.path.dirname(os.getcwd())

In [4]:
#### Data of EAI with CI
data1 = pd.read_excel(os.path.join(path, 'eac_ci_df.xlsx'), index_col = 0)
data1 = data1.loc[:,0:2]
data1 = data1
data1

,0,1,2
0,25.408750,7.179986,5.495185
1,9.760083,3.525186,2.384388
2,7.379005,1.700874,0.976995


In [5]:
#### Data of EAI with CI
data2 = pd.read_excel(os.path.join(path, 'eac_noci_df.xlsx'), index_col = 0)
data2 = data2.loc[:,0:2]
data2 = data2
data2

,0,1,2
0,5.495185,5.495185,5.495185
1,2.384388,2.384388,2.384388
2,0.976995,0.976995,0.976995


In [6]:
data3 = data1 - data2

In [7]:
data3

,0,1,2
0,19.913564,1.684800,0.0
1,7.375695,1.140798,0.0
2,6.402010,0.723879,0.0


In [8]:
benefits = data3.melt()

In [9]:
benefits

,variable,value
0,0,19.913564
1,0,7.375695
2,0,6.402010
3,1,1.684800
4,1,1.140798
5,1,0.723879
6,2,0.000000
7,2,0.000000
8,2,0.000000


#### Equivalent annual cost

#### Discount rate of 2.5%

In [10]:
ops = [1.0, 1.01, 1.025]
tfs = [0, 0.25, 1]
ci  = [0,1,2]

eac_df_r1 = pd.DataFrame(index = [0,1,2], columns = [0,1,2])

for i in range(len(ops)):
    for j in range(len(ci)):
        op = ops[i]
        tf = tfs[i]
        cis = ci[j]
        r = 0.025
        n = 15
        maintenance = 0.2

        eac_df_r1.loc[i,j] = investment_cost_calculator(r , n, maintenance, op, tf, cis)

In [11]:
eac_df_r1 

,0,1,2
0,0.493093,0.350193,0.0
1,0.493093,0.350193,0.0
2,0.493093,0.350193,0.0


#### Discount rate of 5.5%

In [12]:
ops = [1.0, 1.01, 1.025]
tfs = [0, 0.25, 1]
ci  = [0,1,2]

eac_df_r2 = pd.DataFrame(index = [0,1,2], columns = [0,1,2])

for i in range(len(ops)):
    for j in range(len(ci)):
        op = ops[i]
        tf = tfs[i]
        cis = ci[j]
        r = 0.055
        n = 15
        maintenance = 0.2

        eac_df_r2.loc[i,j] = investment_cost_calculator(r , n, maintenance, op, tf, cis)

In [13]:
eac_df_r2

,0,1,2
0,0.608231,0.431964,0.0
1,0.608231,0.431964,0.0
2,0.608231,0.431964,0.0


In [14]:
cost1 = eac_df_r1.melt()
cost1

,variable,value
0,0,0.493093
1,0,0.493093
2,0,0.493093
3,1,0.350193
4,1,0.350193
5,1,0.350193
6,2,0.0
7,2,0.0
8,2,0.0


In [15]:
cost2 = eac_df_r2.melt()
cost2

,variable,value
0,0,0.608231
1,0,0.608231
2,0,0.608231
3,1,0.431964
4,1,0.431964
5,1,0.431964
6,2,0.0
7,2,0.0
8,2,0.0


In [16]:
benefits.to_excel('benefits.xlsx')
cost1.to_excel('cost1.xlsx')
cost2.to_excel('cost2.xlsx')